<a href="https://colab.research.google.com/github/bellaa-a/Hackathon-2024/blob/main/MedalDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:


user_input = input("run model? [y/n]")
# Clone the YOLOv5 repository and install dependencies

if user_input == 'y':
  # Upload your 'best.pt' model file
  from google.colab import files

  # Import necessary libraries
  from torch._C import *
  from pathlib import Path

  # Define the model path
  model_path = '/content/Machine_learning/trained_model/best.pt'

  # Verify if the model file exists
  if not Path(model_path).exists():
      raise FileNotFoundError(f'{model_path} not found. Please upload your model file.')

  # Run inference on an image or video
  # Upload an image or video
  uploaded = ['/content/Machine_learning/data_for_testing/reaching.MOV']

  # List uploaded files
  for filename in uploaded:
      print(f'User uploaded file "{filename}"')

      # Perform inference using YOLOv5
      !python detect.py --weights {model_path} --save-txt --img 640 --conf 0.25 --source {filename} --project {'/content/Machine_learning/results'}


run model? [y/n]y
User uploaded file "/content/Machine_learning/data_for_testing/reaching.MOV"
detect: weights=['/content/Machine_learning/trained_model/best.pt'], source=/content/Machine_learning/data_for_testing/reaching.MOV, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/Machine_learning/results, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-367-gc33abbb1 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/286) /content/Machine_learning/data_for_testing/reaching.MOV: 640x384 2 red medals, 657.1ms
video 1/1 (2/286) /content/Machine_learning/data_for_testing/re

In [146]:
#organize label data
import os
import csv

# Directory containing text files
directory = '/content/Machine_learning/results/exp8/labels'

# Output CSV file
output_csv = '/content/Analyze_shake/coordinates_reaching.csv'

# List to hold extracted coordinates
coordinates = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)

        # Read the entire file content
        with open(filepath, 'r') as file:
            lines = file.readlines()

            if len(lines) >= 1:
                # Process the first line for date and coordinates
                first_line = lines[0].strip()
                parts = first_line.split()

                if len(parts) >= 3:
                    try:
                        # Extract the date (if needed) and the coordinates
                        x_coord = float(parts[1])
                        y_coord = float(parts[2])
                        coordinates.append([x_coord, y_coord])
                    except ValueError:
                        print(f"Skipping line due to value error: {first_line}")

# Write extracted coordinates to CSV
with open(output_csv, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['X Coordinate', 'Y Coordinate'])
    csvwriter.writerows(coordinates)

print(f"Data has been written to {output_csv}")


Data has been written to /content/Analyze_shake/coordinates_reaching.csv


In [149]:
#detect shaking
import pandas as pd
import numpy as np
from google.colab import files

# Step 1: Upload the CSV file
coordinates = '/content/Analyze_shake/coordinates_reaching.csv'

# Step 2: Read the CSV file into a Pandas DataFrame
df = pd.read_csv(coordinates)

frame_rate = 7 #frames per half second

# Step 3: Define the shakiness detection function
#threshold_higher is used to eliminate

def is_shaking(positions, frame_num, threshold_lower =0.01, threshold_higher = 0.025):
    """
    Determine if an object is shaking based on the variance in its positions.

    :param positions: List of (x, y) positions.
    :param threshold: Variance threshold to classify as shaking.
    :return: True if shaking, False otherwise.
    """
    if len(positions) < 2:
        return False

    # Calculate variance in positions
    positions = np.array(positions)
    half_second_movement = positions[[frame_num, frame_num + frame_rate]]
    mean_position = np.mean(half_second_movement, axis=0)
    variance = np.mean(np.linalg.norm(half_second_movement - mean_position, axis=1))
    return variance > threshold_lower and variance < threshold_higher

num_is_shaky = 0;
# Step 4: Extract positions from the DataFrame and check for shakiness
cur_frame = 0;
while cur_frame + frame_rate <= len(df):
  positions = df[['X Coordinate', 'Y Coordinate']].values
  is_shaky = is_shaking(positions, cur_frame)
  #print(is_shaky)

  if is_shaky:
    num_is_shaky += 1
  elif num_is_shaky > 0:
    num_is_shaky -= 1

  if num_is_shaky >= 6:
    print("The doctor is shaking.")
  else:
    print("The doctor is okay.")

  cur_frame+=frame_rate;


0.009155472803192646
0.19173023748616175
0.22117356687678574
0.008907984971361362
0.38329290935164717
0.20567692219838374
0.16343157435605887
0.19806894325777075
0.1838260428863386
0.42496642497172643
0.016731346277272498
0.017746367071882617
0.006528444263375491
0.007887226445335509
0.29901027765488264
0.02350083479900233
0.32971348222396063
0.2162421863929423
0.04252538165025682
0.1457964687929718
0.21246760176200985
0.200781
0.20560643261398703
0.18755376818461955
0.0008682000057590273
0.0007066544417181449
0.00048103975927151106
0.01160929207574691
0.012188347437204122
0.16790424046610614
0.16430328893923576
0.23404399428366024
0.23603090765617962
0.004710310525857062
0.32121360490840045
0.053359763609858696
0.3522620941745649
0.3258087053011322
0.32305221039523935
0.27383537528823776
